## Import libraries

In [1]:
import numpy as np
import os
import sys
import tensorflow as tf
import pprint
import keras


Using TensorFlow backend.


## Data generator 

Keras does not have a generator that reads h5.
Pytorch handles h5 files.

In [2]:
import h5py
from keras.preprocessing.image import ImageDataGenerator

import sklearn.feature_extraction.image as ski


# create patches from slices
def create_patches(x1,params):
  
    x2 = np.zeros((params.batch_size,params.psize[0],params.psize[1],params.nch),dtype='float')
  
    #x1 = x1[:,:params.window[0],:params.window[1],:]

    for i1 in np.arange(params.batch_size):
        x2[i1,:,:,0] = ski.extract_patches_2d(x1[i1,:,:,:],(params.psize[0],params.psize[1]),max_patches=params.npt)

        x2[i1,:,:,0] -= np.min(x2[i1,:,:,0])
        if np.max(x2[i1,:,:,0]) > 0:
            x2[i1,:,:,0] /= np.max(x2[i1,:,:,0])

    #normalize the data
    x2[:,:,:,1] = x2[:,:,:,0]
    x2[:,:,:,2] = x2[:,:,:,0]
    x2 -= .5
    x2 *= 2
    
    return x2

def generate_batches(files, idxt, params):
    nn = len(files)
    counter = 0
    cnt = 0
    
    # create generator to transform images: brightness and rotation
    datagen1 = ImageDataGenerator(brightness_range=[0.2,1.5],rotation_range=16)

    while True:

        hdf5_file1 = h5py.File(files[counter], "r")

        bs = int(params.batch_size/params.npt)

        np.random.shuffle(idxt)

        # loop to repeat the image transformations ntrans times
        for nt in np.arange(params.ntrans):

            for cbatch in np.arange(0,idxt.shape[0],bs):
                idx = idxt[cbatch:(cbatch+bs)]
                idx.sort()
                # read the image and labels in batch sizes
                x1 = (hdf5_file1["img"][idx,:,:,:])
                y1 = (hdf5_file1["labels"][idx,:2])

                # perform the transformation
                if nt > 0:
                    it = datagen1.flow(x1, y1, batch_size=bs,shuffle=True)
                    x1,y1 = it.next()
                #create patches 128x128
                x11 = create_patches(x1,params)

                yield (x11,y1)

        hdf5_file1.close()

        counter += 1
        if counter == nn:
            counter = 0 
    return

## Deep CNN classifier using keras
Using a pre-trained ResNet50


In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys, argparse
os.environ['KERAS_BACKEND'] = 'tensorflow'

from keras.models import Model,load_model
from keras.layers import Dense,Input,Activation
from keras.regularizers import l1

from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau

from keras.optimizers import RMSprop, SGD, Adam
from keras_applications.resnet_v2 import ResNet50V2
from keras.preprocessing.image import ImageDataGenerator

import keras.backend as K

In [4]:
from sklearn.model_selection import StratifiedKFold

def run_CNN(params):

    f = 0
    gen_train = generate_batches(files=[params.input+"train.hdf5"], idxt=np.arange(27430), params=params)
    gen_val = generate_batches(files=[params.input+"val.hdf5"], idxt=np.arange(5880),params=params)       


    model,model_name,loadmdl = MyModel(params,f)

    if loadmdl == 0:
            print('sgd = Adam(lr=3e-3)')
            sgd = Adam(3e-3)

            model = train_CNN(params,model,15,sgd,gen_train,gen_val,model_name)

    else:
            sgd = Adam(3e-5)

            model = train_CNN(params,model,5,sgd,gen_train,gen_val,model_name)

    return

In [5]:
def MyModel(params,f):

    model_name = params.models+str(f)
    if not os.path.isfile(model_name + '.model'):
        print("Creating Model : ", model_name)
        model_base = ResNet50V2(include_top=False, weights='imagenet', input_shape=(params.psize[0], params.psize[1],params.nch), pooling='avg',backend=keras.backend, layers=keras.layers, models=keras.models, utils=keras.utils)

        x = model_base.get_layer('avg_pool').output
        dense = Dense(2,
                    kernel_initializer='he_normal',
                    bias_initializer='zeros',
                    kernel_regularizer=l1(0.01),
                    name='classifier')(x)
  
        prediction = Activation("softmax",name="softmax")(dense)

        model = Model(model_base.inputs, outputs=prediction, name='ResNet')

        loadmdl = 0
    else:
        print("Loading model: ", model_name)
        model = load_model( model_name+'.model')
        loadmdl = 1


    return model,model_name,loadmdl


def train_CNN(params,model,nepochs,sgd,gen_train,gen_val,model_name):


        #print("Compiling model...",model_name)

        model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=["accuracy"])
        
        print("Compiled...") 
        
        model.summary()
        
        filepath=model_name+"epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"

 
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                  patience=3, min_lr=1e-8)

        earlyStopping = EarlyStopping(monitor='val_loss',
                           patience=10, 
                           verbose=1, mode='auto')

        checkpoint = ModelCheckpoint(filepath, monitor='val_loss',
                         verbose=0, save_best_only=True)

        nsamplesperfold = 1.*params.ntrans*params.npt/(params.batch_size)
        #print(nsamplesperfold)

        model.fit_generator(gen_train, steps_per_epoch = (27430*nsamplesperfold),
                  epochs=nepochs, verbose=1,
                  callbacks=[checkpoint,earlyStopping,reduce_lr],
                  validation_data=gen_val, validation_steps = (5880*nsamplesperfold))



        return model

In [ ]:
sys.argv = ['foo']

parser = argparse.ArgumentParser()

parser.add_argument('-input',
                    type=str,
                    default='/home/ubuntu/volume1/BrainHack/data/BrainHack_quality_',
                    help='txt file containing input data filenames')

parser.add_argument('-models', type=str, 
                    default='/home/ubuntu/volume1/BrainHack/models/model_BrainHack_CTMR_h5_',
                    help='models prefix directory')

parser.add_argument('-psize', type=int, default=[128,128],
                    help='patch size extracted from dataset')

parser.add_argument('-window', type=int, default=[224,224],
                    help='area of interest ')

parser.add_argument('-img_size', type=int, default=[224,224],
                    help='area of interest ')

parser.add_argument('-nfolds', type=int, default=1,
                    help='number of folds for training')

parser.add_argument('-nch', type=int, default=3,
                    help='number of input channels')

parser.add_argument('-npt', type=int, default=1,
                    help='the number of patches from slices')

parser.add_argument('-ntrans', type=int, default=5,
                    help='number of transformations ')

parser.add_argument('-batch_size', type=int, default=10,
                    help='size of batch')

params = parser.parse_args()

run_CNN(params)

W1025 17:11:13.044018 139968345294592 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1025 17:11:13.063619 139968345294592 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1025 17:11:13.069634 139968345294592 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1025 17:11:13.086151 139968345294592 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The 

Creating Model :  /home/ubuntu/volume1/BrainHack/models/model_BrainHack_CTMR_h5_0


W1025 17:11:21.803143 139968345294592 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



sgd = Adam(lr=3e-3)
Compiled...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 66, 66, 64)   0           conv1_conv[0][0]                 
_____________________________________________________________________________

W1025 17:11:22.114987 139968345294592 deprecation.py:323] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/15
   13/13715 [..............................] - ETA: 18:48:10 - loss: 2.2837 - acc: 0.6077